In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import load_model
import tensorflow as tf
import joblib
import time

In [2]:
# df = pd.DataFrame(data)
file_path = "../datasets/train_set2.csv"
df = pd.read_csv(file_path)

EMPTYCONST = "*empty*"

# Concatenate relevant text features
catFamily = df["Family"].fillna(EMPTYCONST)
catSubFamily = df["SubFamily"].fillna(EMPTYCONST)
catObjectGroup = df["ObjectGroup"].fillna(EMPTYCONST)
catObjectName = df["ObjectName"].fillna(EMPTYCONST)
catDescription = df["Description"].fillna(EMPTYCONST)
catTypeComments = df["Type Comments"].fillna(EMPTYCONST)
catStructuralMaterial = df["Structural Material"].fillna(EMPTYCONST)
catMaterial = df["Material"].fillna(EMPTYCONST)


df["Features"] = (
    catFamily
    + "|"
    + catSubFamily
    + "|"
    + catObjectGroup
    + "|"
    + catObjectName
    + "|"
    + catDescription
    + "|"
    + catTypeComments
    + "|"
    + catStructuralMaterial
    + "|"
    + catMaterial
)

# Start timing
start_time = time.time()

# Encode categorical variable 'Category'
label_encoder = LabelEncoder()
df['Category'] = label_encoder.fit_transform(df['Category'])

# Split data into features (X) and target variable (y)
X = df["Features"].fillna(EMPTYCONST)
y = df["Category"]

# Vectorize text data using TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

# Convert X_tfidf to a dense numpy array
X_tfidf_dense = X_tfidf.toarray()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf_dense, y, test_size=0.2, random_state=42
)

# Define the neural network model
model_nn = Sequential([
    Dense(units=512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(units=256, activation='relu'),
    Dropout(0.5),
    Dense(units=64, activation='sigmoid'),
    Dense(len(df['Category'].unique()), activation='softmax')
])


# Compile the model
model_nn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_nn.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# End timing
end_time = time.time()

# Calculate total runtime
total_time = end_time - start_time
print(f"Total runtime: {total_time} seconds")

# Evaluate the model on the testing set
loss, accuracy = model_nn.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


# Save the model to disk
model_nn.save('FNN_model_tif.h5')
print("Model saved to disk.")
# print("\nConfusion Matrix:")
# print(confusion_matrix(y_test, predictions))

Epoch 1/50
65/65 [==============================] - 2s 12ms/step - loss: 3.0658 - accuracy: 0.2103 - val_loss: 2.5363 - val_accuracy: 0.2208
Epoch 2/50
65/65 [==============================] - 1s 10ms/step - loss: 2.5497 - accuracy: 0.2132 - val_loss: 2.4891 - val_accuracy: 0.2208
Epoch 3/50
65/65 [==============================] - 1s 10ms/step - loss: 2.5361 - accuracy: 0.2233 - val_loss: 2.4961 - val_accuracy: 0.2035
Epoch 4/50
65/65 [==============================] - 1s 10ms/step - loss: 2.5171 - accuracy: 0.2123 - val_loss: 2.4428 - val_accuracy: 0.2208
Epoch 5/50
65/65 [==============================] - 1s 10ms/step - loss: 2.5185 - accuracy: 0.2233 - val_loss: 2.4365 - val_accuracy: 0.2035
Epoch 6/50
65/65 [==============================] - 1s 9ms/step - loss: 2.5128 - accuracy: 0.2277 - val_loss: 2.4858 - val_accuracy: 0.2035
Epoch 7/50
65/65 [==============================] - 1s 10ms/step - loss: 2.5234 - accuracy: 0.2171 - val_loss: 2.4395 - val_accuracy: 0.2035
Epoch 8/50
65/

In [3]:
model = load_model('FNN_model_tif.h5')
# Load new data (example: assume `new_data.csv` has the same structure as `train_set.csv`)
new_data_file = "../datasets/test_set.csv"
new_df = pd.read_csv(new_data_file)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),  # or any other optimizer you want to use
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# Concatenate relevant text features
catFamily = new_df["Family"].fillna("*empty*")
catSubFamily = new_df["SubFamily"].fillna("*empty*")
catObjectGroup = new_df["ObjectGroup"].fillna("*empty*")
catObjectName = new_df["ObjectName"].fillna("*empty*")
catDescription = new_df["Description"].fillna("*empty*")
catTypeComments = new_df["Type Comments"].fillna("*empty*")
catStructuralMaterial = new_df["Structural Material"].fillna("*empty*")
catMaterial = new_df["Material"].fillna("*empty*")

new_df["Features"] = (
    catFamily
    + "|"
    + catSubFamily
    + "|"
    + catObjectGroup
    + "|"
    + catObjectName
    + "|"
    + catDescription
    + "|"
    + catTypeComments
    + "|"
    + catStructuralMaterial
    + "|"
    + catMaterial
)

test2343 = label_encoder.transform(new_df['Category'])


# Vectorize text data using the same CountVectorizer instance
X_new_tfidf = vectorizer.transform(new_df["Features"].fillna(EMPTYCONST))
# Convert X_tfidf to a dense numpy array
X_tfidf_new_dense = X_new_tfidf.toarray()

# Predict using the loaded model
predictions_new = model_nn.predict(X_tfidf_new_dense)
predicted_classes = predictions_new.argmax(axis=1)  # Assuming predictions_new is one-hot encoded

# Calculate accuracy
accuracy = accuracy_score(test2343, predicted_classes)
print("Accuracy:", accuracy)


4/4 [==============================] - 0s 2ms/step
Accuracy: 0.0
